In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig
from torch.optim.lr_scheduler import StepLR

# 自定义数据集类
class CustomDataset(Dataset):
    def __init__(self, data_path, tokenizer):
        self.sentences, self.labels = self.load_data(data_path)
        self.tokenizer = tokenizer

    def load_data(self, data_path):
        sentences = []
        labels = []
        with open(data_path, 'r', encoding='utf-8') as file:
            for line in file:
                sentence, label = line.strip().split('&')
                sentences.append(sentence)
                labels.append(int(label))
        return sentences, labels

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        label = self.labels[idx]

        # 使用BERT的tokenizer对句子进行编码
        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label': torch.tensor(label)
        }


# 设置训练参数
data_path = './data.txt'
model_name = 'hfl/chinese-roberta-wwm-ext'
batch_size = 32
label_cnt = 7
learning_rate = 5e-6
num_epochs = 27

# 加载预训练的BERT模型和tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name)
config = BertConfig.from_pretrained(model_name)
config.num_hidden_layers = 2
config.num_labels = label_cnt   # 将 num_labels 添加到 config 中
model = BertForSequenceClassification.from_pretrained(model_name, config=config)
# model.classifier.add_module('dropout', torch.nn.Dropout(p=0.5))
# model.classifier.add_module('fc1', torch.nn.Linear(768, 256))  # 在原有的分类器上增加一个全连接层
# model.classifier.add_module('fc2', torch.nn.Linear(256, 128)) # 再加一层
# model.classifier.add_module('fc3', torch.nn.Linear(128, 7))


Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertForSequenceClassification: ['bert.encoder.layer.2.intermediate.dense.bias', 'bert.encoder.layer.6.output.LayerNorm.weight', 'bert.encoder.layer.6.attention.self.query.weight', 'bert.encoder.layer.7.attention.self.key.bias', 'bert.encoder.layer.5.output.LayerNorm.bias', 'bert.encoder.layer.4.intermediate.dense.weight', 'bert.encoder.layer.11.attention.self.value.bias', 'cls.predictions.transform.LayerNorm.bias', 'bert.encoder.layer.9.attention.output.dense.weight', 'bert.encoder.layer.5.attention.output.dense.weight', 'bert.encoder.layer.9.output.dense.bias', 'bert.encoder.layer.6.output.LayerNorm.bias', 'bert.encoder.layer.10.intermediate.dense.bias', 'bert.encoder.layer.4.attention.output.dense.weight', 'bert.encoder.layer.9.attention.output.LayerNorm.weight', 'bert.encoder.layer.10.attention.self.query.weight', 'bert.encoder.layer.8.intermediate.dense.bias', 'cls.predictions.transf

In [5]:
# 加载数据集
dataset = CustomDataset(data_path, tokenizer)

# 划分训练集和验证集
train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(
    dataset, [train_size, valid_size])

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)

# 定义优化器和损失函数
# weight_decay为L2正则化 防止过拟合
# scheduler 配置学习率衰减
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0.0001)
scheduler = StepLR(optimizer, step_size=18, gamma=0.1)
loss_fn = torch.nn.CrossEntropyLoss()

c:\Users\12236\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.device_count() > 1:
    model = torch.DataParallel(model, device_ids=[0, 1, 2])

model.to(device)
print(device)

cuda


In [7]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        _, predicted_labels = torch.max(outputs.logits, dim=1)
        correct_predictions += (predicted_labels == labels).sum().item()
        total_predictions += labels.size(0)
    
    # 在验证集上评估模型
    model.eval()
    valid_loss = 0
    correct_valid_predictions = 0
    total_valid_predictions = 0

    with torch.no_grad():
        for batch in valid_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            _, predicted_labels = torch.max(logits, dim=1)

            valid_loss += loss_fn(logits, labels).item()
            correct_valid_predictions += (predicted_labels == labels).sum().item()
            total_valid_predictions += labels.size(0)

    epoch_loss = total_loss / len(train_loader)
    epoch_valid_loss = valid_loss / len(valid_loader)
    accuracy = correct_predictions / total_predictions
    valid_accuracy = correct_valid_predictions / total_valid_predictions
    scheduler.step()

    print(f'Epoch {epoch + 1}/{num_epochs} - Loss: {epoch_loss:.4f} - Valid Loss: {epoch_valid_loss:.4f} - Accuracy: {accuracy:.4f} - Valid Accuracy: {valid_accuracy:.4f}')

Epoch 1/27 - Loss: 1.7449 - Valid Loss: 1.5058 - Accuracy: 0.3494 - Valid Accuracy: 0.4929
Epoch 2/27 - Loss: 1.1859 - Valid Loss: 0.8354 - Accuracy: 0.5962 - Valid Accuracy: 0.7333
Epoch 3/27 - Loss: 0.6783 - Valid Loss: 0.5053 - Accuracy: 0.8048 - Valid Accuracy: 0.8333
Epoch 4/27 - Loss: 0.4465 - Valid Loss: 0.3816 - Accuracy: 0.8765 - Valid Accuracy: 0.8778
Epoch 5/27 - Loss: 0.3399 - Valid Loss: 0.3306 - Accuracy: 0.9065 - Valid Accuracy: 0.8873
Epoch 6/27 - Loss: 0.2820 - Valid Loss: 0.2957 - Accuracy: 0.9226 - Valid Accuracy: 0.8984
Epoch 7/27 - Loss: 0.2483 - Valid Loss: 0.2709 - Accuracy: 0.9267 - Valid Accuracy: 0.9040
Epoch 8/27 - Loss: 0.2220 - Valid Loss: 0.2568 - Accuracy: 0.9347 - Valid Accuracy: 0.9079
Epoch 9/27 - Loss: 0.1912 - Valid Loss: 0.2465 - Accuracy: 0.9436 - Valid Accuracy: 0.9127
Epoch 10/27 - Loss: 0.1809 - Valid Loss: 0.2448 - Accuracy: 0.9454 - Valid Accuracy: 0.9127
Epoch 11/27 - Loss: 0.1630 - Valid Loss: 0.2322 - Accuracy: 0.9510 - Valid Accuracy: 0.91

In [8]:
# 计算整体准确率
v_loader = DataLoader(dataset)
model.eval()
correct_predictions = 0
total_predictions = 0
with torch.no_grad():
        for batch in v_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            _, predicted_labels = torch.max(logits, dim=1)

            correct_predictions += (predicted_labels == labels).sum().item()
            total_predictions += labels.size(0)

valid_accuracy = correct_predictions / total_predictions

print(valid_accuracy)

0.9780848022868033


In [9]:
# 保存模型
torch.save(model.state_dict(), 'Roberta3.pt')

In [2]:
def predict_label(sentence, model, tokenizer):
    # 对输入句子进行编码
    encoding = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].squeeze()
    attention_mask = encoding['attention_mask'].squeeze()

    # 在模型中进行前向传播
    with torch.no_grad():
        outputs = model(input_ids.unsqueeze(0), attention_mask=attention_mask.unsqueeze(0))
        logits = outputs.logits

    # 获取预测的标签
    _, predicted_label = torch.max(logits, dim=1)

    return predicted_label.item()

# # 加载训练好的模型参数
model.load_state_dict(torch.load('Roberta3.pt'))

# 设置模型为评估模式
model.eval()

# 测试例子
input_sentence = '2016.9-2019.7          广州二中           高中'
predicted_label = predict_label(input_sentence, model, tokenizer)

print(f'输入句子: {input_sentence}')
print(f'预测标签: {predicted_label}')


输入句子: 2016.9-2019.7          广州二中           高中
预测标签: 0


In [ ]:
label = {
    0: '基本信息',
    1: '求职意向',
    2: '工作/项目经历',
    3: '获得奖项',
    4: '个人能力',
    5: '垃圾信息',
    6: '重要标注'
}
sentences = [
    '黄岩新前卫生院',
    '广州市第三人民医院',
    '广州医科大学',
    '黄岩新前卫生院',
    '广州铁塔第一医院',
    '北京协和医院',
    '上海交通大学医学院附属瑞金医院',
    '广州中山大学附属第一医院',
    '成都华西医院',
    '武汉大学人民医院',
    '复旦大学附属华山医院',
    '中山大学附属第三医院',
    '南京大学医学院附属鼓楼医院',
    '重庆医科大学附属第一医院',
    '浙江大学医学院附属第一医院',
    '2009.09-2013.06 复旦大学 环境科学',
    '2013.09-2017.06 吉林大学 旅游管理',
    '4-6k',
    '8-10k',
    '1-3k',
    '4-5w',
    '1-7u',
    '6-9p',
    '许爱礼',
    '陈虹荣',
    '陈虹荣 求职意向：工商管理',
    '胡健 求职意向：健身教练',
    '33',
    '34',
    '林竹水',
    '王爱乐',
    '2017.06-至今       安宁家乐福超市',
    '2015.09-2015.12	 兰州大学食堂咖啡店',
    '实习护士',
    '临床护士',
    '管理学院',
    '韩健毓'
]

for s in sentences:
    p = predict_label(s, model, tokenizer)
    print(f'{s} -- {label[p]}')


黄岩新前卫生院 -- 工作/项目经历
广州市第三人民医院 -- 工作/项目经历
广州医科大学 -- 基本信息
黄岩新前卫生院 -- 工作/项目经历
广州铁塔第一医院 -- 工作/项目经历
北京协和医院 -- 工作/项目经历
上海交通大学医学院附属瑞金医院 -- 工作/项目经历
广州中山大学附属第一医院 -- 工作/项目经历
成都华西医院 -- 工作/项目经历
武汉大学人民医院 -- 工作/项目经历
复旦大学附属华山医院 -- 工作/项目经历
中山大学附属第三医院 -- 工作/项目经历
南京大学医学院附属鼓楼医院 -- 工作/项目经历
重庆医科大学附属第一医院 -- 工作/项目经历
浙江大学医学院附属第一医院 -- 工作/项目经历
2009.09-2013.06 复旦大学 环境科学 -- 基本信息
2013.09-2017.06 吉林大学 旅游管理 -- 基本信息
4-6k -- 垃圾信息
8-10k -- 垃圾信息
1-3k -- 垃圾信息
4-5w -- 垃圾信息
1-7u -- 垃圾信息
6-9p -- 垃圾信息
许爱礼 -- 基本信息
陈虹荣 -- 基本信息
陈虹荣 求职意向：工商管理 -- 基本信息
胡健 求职意向：健身教练 -- 求职意向
33 -- 垃圾信息
34 -- 垃圾信息
林竹水 -- 基本信息
王爱乐 -- 基本信息
2017.06-至今       安宁家乐福超市 -- 工作/项目经历
2015.09-2015.12	 兰州大学食堂咖啡店 -- 工作/项目经历
实习护士 -- 基本信息
临床护士 -- 基本信息
管理学院 -- 垃圾信息
